In [1]:
## https://github.com/ydataai/ydata-synthetic/blob/dev/examples/timeseries/TimeGAN_Synthetic_stock_data.ipynb

from tensorflow.keras import Input, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanAbsoluteError

from data_hub import LoadData

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_log_error
import pandas as pd

import numpy as np

2022-05-16 17:04:04.006225: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
def RNN_regression(units):
    opt = Adam(name='AdamOpt')
    loss = MeanAbsoluteError(name='MAE')
    model = Sequential()
    model.add(GRU(units=units,
                  name=f'RNN_1'))
    model.add(Dense(units=7,
                    activation='sigmoid',
                    name='OUT'))
    model.compile(optimizer=opt, loss=loss)
    return model

In [3]:
dataset_name = 'sports-goal'
seq_len = 24

In [4]:
train_data, test_data = LoadData(dataset_name, seq_len)

Loading Saved data
Sports Data with 7 dimension loaded with sequence 24


In [5]:
synth_data = np.load('synth_data/synth_data_timegan.npy')

In [6]:
ori_data=np.asarray(test_data)

synth_data = synth_data[:len(ori_data)]

n_events = len(ori_data)


idx = np.arange(n_events)

n_train = int(.75*n_events)

train_idx = idx[:n_train]
test_idx = idx[n_train:]


X_ori_train = ori_data[train_idx, :seq_len-1, :]
X_synth_train = synth_data[train_idx, :seq_len-1, :]

X_ori_test = ori_data[test_idx, :seq_len-1, :]
y_ori_test = ori_data[test_idx, -1, :]


y_ori_train = ori_data[train_idx, -1, :]
y_synth_train = synth_data[train_idx, -1, :]

print('Synthetic X train: {}'.format(X_synth_train.shape))
print('Real X train: {}'.format(X_ori_train.shape))

print('Synthetic y train: {}'.format(y_synth_train.shape))
print('Real y train: {}'.format(y_ori_train.shape))

print('Real X test: {}'.format(X_ori_test.shape))
print('Real y test: {}'.format(y_ori_test.shape))

Synthetic X train: (489, 23, 7)
Real X train: (489, 23, 7)
Synthetic y train: (489, 7)
Real y train: (489, 7)
Real X test: (164, 23, 7)
Real y test: (164, 7)


In [7]:
ts_synth = RNN_regression(12)
early_stopping = EarlyStopping(monitor='val_loss')
synth_train = ts_synth.fit(x=X_synth_train,
                          y=y_synth_train,
                          validation_data=(X_ori_test, y_ori_test),
                          epochs=200,
                          batch_size=128,
                          callbacks=[early_stopping])

2022-05-16 17:04:05.369671: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-16 17:04:05.371194: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-16 17:04:05.408822: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:15:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-05-16 17:04:05.410605: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:21:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2022-05-16 17:04:05.410659: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart

Epoch 1/200


2022-05-16 17:04:07.070120: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11


3/4 [=====================>........] - ETA: 0s - loss: 0.2957

2022-05-16 17:04:07.395823: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


4/4 [==============================] - 2s 157ms/step - loss: 0.2969 - val_loss: 0.2841
Epoch 2/200
4/4 [==============================] - 0s 66ms/step - loss: 0.2958 - val_loss: 0.2818
Epoch 3/200
4/4 [==============================] - 0s 63ms/step - loss: 0.2937 - val_loss: 0.2797
Epoch 4/200
4/4 [==============================] - 0s 65ms/step - loss: 0.2904 - val_loss: 0.2775
Epoch 5/200
4/4 [==============================] - 0s 64ms/step - loss: 0.2884 - val_loss: 0.2754
Epoch 6/200
4/4 [==============================] - 0s 56ms/step - loss: 0.2860 - val_loss: 0.2735
Epoch 7/200
4/4 [==============================] - 0s 69ms/step - loss: 0.2853 - val_loss: 0.2716
Epoch 8/200
4/4 [==============================] - 0s 77ms/step - loss: 0.2828 - val_loss: 0.2697
Epoch 9/200
4/4 [==============================] - 0s 66ms/step - loss: 0.2792 - val_loss: 0.2678
Epoch 10/200
4/4 [==============================] - 0s 64ms/step - loss: 0.2776 - val_loss: 0.2661
Epoch 11/200
4/4 [============

In [8]:
synth_predictions = ts_synth.predict(X_ori_test)

print(f'MAE: {mean_absolute_error(y_ori_test, synth_predictions)}')
print(f'MRLE: {mean_squared_log_error(y_ori_test, synth_predictions)}')

MAE: 0.24017944931983948
MRLE: 0.05028403550386429
